# Basic Metrics

When we think about summarizing data, what are the metrics that we look at?

In this notebook, we will look in the price of weed dataset along with the demographic information of the United States. 

To read how the data was acquired, please read [this](https://github.com/amitkaps/weed/blob/master/1-Acquire.ipynb) to get more information

This notebook will make use of pandas quite a bit.

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from scipy import stats

### Read the input datasets. There are three datasets:

1. Weed price by date / state
2. Demographics of State
3. Population of state

In [4]:
prices_pd = pd.read_csv("../data/Weed_Price.csv", parse_dates=[-1])
demography_pd = pd.read_csv("../data/Demographics_State.csv")
population_pd = pd.read_csv("../data/Population_State.csv")

In [3]:
prices_pd.head()

State   HighQ  HighQN    MedQ  MedQN    LowQ  LowQN       date
0     Alabama  339.06    1042  198.64    933  149.49    123 2014-01-01
1      Alaska  288.75     252  260.60    297  388.58     26 2014-01-01
2     Arizona  303.31    1941  209.35   1625  189.45    222 2014-01-01
3    Arkansas  361.85     576  185.62    544  125.87    112 2014-01-01
4  California  248.78   12096  193.56  12812  192.92    778 2014-01-01

In [4]:
prices_pd.tail()

State   HighQ  HighQN    MedQ  MedQN  LowQ  LowQN       date
22894       Virginia  364.98    3513  293.12   3079   NaN    284 2014-12-31
22895     Washington  233.05    3337  189.92   3562   NaN    160 2014-12-31
22896  West Virginia  359.35     551  224.03    545   NaN     60 2014-12-31
22897      Wisconsin  350.52    2244  272.71   2221   NaN    167 2014-12-31
22898        Wyoming  322.27     131  351.86    197   NaN     12 2014-12-31

In [5]:
demography_pd.head()

region  total_population  percent_white  percent_black  percent_asian  \
0     alabama           4799277             67             26              1   
1      alaska            720316             63              3              5   
2     arizona           6479703             57              4              3   
3    arkansas           2933369             74             15              1   
4  california          37659181             40              6             13   

   percent_hispanic  per_capita_income  median_rent  median_age  
0                 4              23680          501        38.1  
1                 6              32651          978        33.6  
2                30              25358          747        36.3  
3                 7              22170          480        37.5  
4                38              29527         1119        35.4

In [6]:
population_pd.head()

region     value
0     alabama   4777326
1      alaska    711139
2     arizona   6410979
3    arkansas   2916372
4  california  37325068

In [7]:
prices_pd.dtypes

State             object
HighQ            float64
HighQN             int64
MedQ             float64
MedQN              int64
LowQ             float64
LowQN              int64
date      datetime64[ns]
dtype: object

#### Sort the data on state and date, then fill NA values

In [ ]:
prices_pd.sort(columns=['State', 'date'], inplace=True)
prices_pd.fillna(method='ffill', inplace=True)

### Finding mean, median, mode, variance, standard deviation for California

#### Mean

arithmetic average of a range of values or quantities, computed by dividing the total of all values by the number of values.

In [10]:
california_pd = prices_pd[prices_pd.State == "California"].copy(True)
california_pd.head()

State   HighQ  HighQN    MedQ  MedQN    LowQ  LowQN       date
4    California  248.78   12096  193.56  12812  192.92    778 2014-01-01
55   California  243.96   16512  189.35  19151     NaN   1096 2015-01-01
106  California  248.20   12571  192.80  13406  191.94    804 2014-02-01
157  California  243.30   16904  188.95  19764     NaN   1123 2015-02-01
208  California  247.60   12988  192.97  13906  191.40    839 2014-03-01

In [11]:
ca_sum = california_pd['HighQ'].sum()

In [12]:
ca_count = california_pd['HighQ'].count()

In [13]:
ca_mean = ca_sum / ca_count
print("Mean weed price in CA is:", ca_mean)

Mean weed price in CA is: 245.37612472160356


#### Exercise: Find CA mean for 2013, 2014 & 2015 separately

*Hint:* `california_pd.iloc[0]['date'].year`

In [14]:
california_pd.iloc[0]['date'].year
california_pd.iloc[1]['date'].year

2014

#### Median

Denotes value or quantity lying at the midpoint of a frequency distribution of observed values or quantities, such that there is an equal probability of falling above or below it. Simply put, it is the *middle* value in the list of numbers.

In [13]:
ca_count

449

If count is odd, the median is the value at (n+1)/2,

else it is the average of n/2 and (n+1)/2

In [ ]:
ca_highq_pd = california_pd.sort(columns=['HighQ'])
ca_highq_pd.head()

In [15]:
ca_median = ca_highq_pd.HighQ.iloc[(ca_count) / 2]
print "Median price of weed in CA is:", ca_median

Median price of weed in CA is: 245.31


#### Mode

It is the number which appears most often in a set of numbers. 

In [16]:
ca_mode = ca_highq_pd.HighQ.value_counts().index[0]
print "The most common price is CA, as indicated by its mode, is:", ca_mode

The most common price is CA, as indicated by its mode, is: 245.05


#### Variance

> Once two statistician of height 4 feet and 5 feet have to cross a river of AVERAGE depth 3 feet. Meanwhile, a third person comes and said, "what are you waiting for? You can easily cross the river"

It's the average distance of the data values from the *mean*

<img style="float: left;" src="img/variance.png" height="320" width="320">

In [17]:
california_pd['HighQ_dev'] = (california_pd['HighQ'] - ca_mean) ** 2

In [18]:
ca_HighQ_variance = california_pd.HighQ_dev.sum() / (ca_count - 1)
print "Variance of High Quality weed prices in CA is:", ca_HighQ_variance

Variance of High Quality weed prices in CA is: 2.98268628798


#### Standard Deviation

It is the square root of variance. This will have the same units as the data and mean. 

In [19]:
ca_HighQ_SD = np.sqrt(ca_HighQ_variance)
print "Standard Deviation of High Quality weed prices in CA is:", ca_HighQ_SD

Standard Deviation of High Quality weed prices in CA is: 1.72704553732


#### Using Pandas built-in function

In [20]:
california_pd.describe()

HighQ        HighQN        MedQ         MedQN        LowQ  \
count  449.000000    449.000000  449.000000    449.000000  449.000000   
mean   245.376125  14947.073497  191.268909  16769.821826  189.783586   
std      1.727046   1656.133565    1.524028   2433.943191    1.598252   
min    241.840000  12021.000000  187.850000  12724.000000  187.830000   
25%    244.480000  13610.000000  190.260000  14826.000000  188.600000   
50%    245.310000  15037.000000  191.570000  16793.000000  188.600000   
75%    246.220000  16090.000000  192.550000  18435.000000  191.320000   
max    248.820000  18492.000000  193.630000  22027.000000  193.880000   

             LowQN   HighQ_dev  
count   449.000000  449.000000  
mean    976.298441    2.976043  
std     120.246714    3.961134  
min     770.000000    0.000015  
25%     878.000000    0.106357  
50%     982.000000    0.729103  
75%    1060.000000    4.435761  
max    1232.000000   12.504178

In [21]:
california_pd.HighQ.mode()

0    245.03
1    245.05
dtype: float64

#### Co-variance 

covariance as a measure of the (average) co-variation between two variables, say x and y. Covariance describes both how far the variables are spread out, and the nature of their relationship, Covariance is a measure of how much two variables change together. Compare this to Variance, which is just the range over which one measure (or variable) varies.

<img style="float: left;" src="img/covariance.png" height="270" width="270">

<br>
<br>
<br>
<br>

#### Co-variance of weed price in California vs New York

In [22]:
ny_pd = prices_pd[prices_pd['State'] == 'New York'].copy(True)
ny_pd.head()

State   HighQ  HighQN    MedQ  MedQN    LowQ  LowQN       date
20120  New York  351.98    5773  268.83   5786  190.31    479 2013-12-27
20885  New York  351.92    5775  268.83   5786  190.31    479 2013-12-28
21599  New York  351.99    5785  269.02   5806  190.75    480 2013-12-29
22313  New York  352.02    5791  268.98   5814  190.75    480 2013-12-30
22823  New York  351.97    5794  268.93   5818  190.75    480 2013-12-31

In [23]:
ny_pd = ny_pd.ix[:,[1,7]]
ny_pd.columns = ['NY_HighQ', 'date']

In [24]:
ny_pd.head()

NY_HighQ       date
20120    351.98 2013-12-27
20885    351.92 2013-12-28
21599    351.99 2013-12-29
22313    352.02 2013-12-30
22823    351.97 2013-12-31

In [25]:
ca_ny_pd = pd.merge(california_pd.ix[:,[1,7]].copy(), ny_pd, on="date")
ca_ny_pd.rename(columns={"HighQ": "CA_HighQ"}, inplace=True)
ca_ny_pd.head()

CA_HighQ       date  NY_HighQ
0    248.77 2013-12-27    351.98
1    248.74 2013-12-28    351.92
2    248.76 2013-12-29    351.99
3    248.82 2013-12-30    352.02
4    248.76 2013-12-31    351.97

In [26]:
ny_mean = ca_ny_pd.NY_HighQ.mean()
ny_mean

346.9127616926502

In [27]:
ca_ny_pd['ca_dev'] = ca_ny_pd['CA_HighQ'] - ca_mean
ca_ny_pd.head()

CA_HighQ       date  NY_HighQ    ca_dev
0    248.77 2013-12-27    351.98  3.393875
1    248.74 2013-12-28    351.92  3.363875
2    248.76 2013-12-29    351.99  3.383875
3    248.82 2013-12-30    352.02  3.443875
4    248.76 2013-12-31    351.97  3.383875

In [28]:
ca_ny_pd['ny_dev'] = ca_ny_pd['NY_HighQ'] - ny_mean
ca_ny_pd.head()

CA_HighQ       date  NY_HighQ    ca_dev    ny_dev
0    248.77 2013-12-27    351.98  3.393875  5.067238
1    248.74 2013-12-28    351.92  3.363875  5.007238
2    248.76 2013-12-29    351.99  3.383875  5.077238
3    248.82 2013-12-30    352.02  3.443875  5.107238
4    248.76 2013-12-31    351.97  3.383875  5.057238

In [29]:
ca_ny_cov = (ca_ny_pd['ca_dev'] * ca_ny_pd['ny_dev']).sum() / (ca_count - 1)
print "Covariance of the High Quality weed prices in CA and NY is:", ca_ny_cov

Covariance of the High Quality weed prices in CA and NY is: 5.91681496729


#### Using Pandas built-in function

In [30]:
ca_ny_pd.cov()

CA_HighQ   NY_HighQ    ca_dev     ny_dev
CA_HighQ  2.982686   5.916815  2.982686   5.916815
NY_HighQ  5.916815  12.245147  5.916815  12.245147
ca_dev    2.982686   5.916815  2.982686   5.916815
ny_dev    5.916815  12.245147  5.916815  12.245147

### Correlation

Extent to which two or more variables fluctuate together. A positive correlation indicates the extent to which those variables increase or decrease in parallel; a negative correlation indicates the extent to which one variable increases as the other decreases.

<img style="float: left;" src="img/correlation.gif" height="270" width="270">

<br>
<br>
<br>



In [31]:
#### Finding correlation between weed prices in New York and California
ca_highq_std = ca_ny_pd.CA_HighQ.std()
ny_highq_std = ca_ny_pd.NY_HighQ.std()

ca_ny_corr = ca_ny_cov / (ca_highq_std * ny_highq_std)
print "Correlation between weed prices in NY and CA:", ca_ny_corr

Correlation between weed prices in NY and CA: 0.979043961106


In [32]:
ca_ny_pd.corr()

CA_HighQ  NY_HighQ    ca_dev    ny_dev
CA_HighQ  1.000000  0.979044  1.000000  0.979044
NY_HighQ  0.979044  1.000000  0.979044  1.000000
ca_dev    1.000000  0.979044  1.000000  0.979044
ny_dev    0.979044  1.000000  0.979044  1.000000

# Correlation != Causation

correlation between two variables does not necessarily imply that one causes the other.


<img style="float: left;" src="img/correlation_not_causation.gif" height="570" width="570">